In [1]:
import sys
sys.path.append('/home/macul/insightface/src/symbols')
sys.path.append('/home/macul/insightface/src/')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/mxiter')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/mxconv')

In [2]:
import fresnet
import mxnet as mx
import numpy as np
import cv2
from collections import namedtuple
import mxresnet
from mxiter import MyImageIter, MyImageIter_IndividualLabel, ImageMultiLabelIter
import json
def get_image(filename):
    img = cv2.imread(filename)  # read image in b,g,r order
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # change to r,g,b order
    img = cv2.resize(img, (112, 112))  # resize to 224*224 to fit model
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)  # change to (channel, height, width)
    img = img[np.newaxis, :]  # extend to (example, channel, heigth, width)
    return img

Batch = namedtuple('Batch', ['data'])

In [3]:
means = json.loads(open( '/media/macul/black/face_database_raw_data/mscelb_from_insightface/mean.json').read())

In [4]:
means

{u'B': 96.73112294438216, u'G': 110.22429227043047, u'R': 138.04845799981138}

In [5]:
train_dataiter = MyImageIter(
        batch_size           = 1,
        data_shape           = (3,112,112),
        path_imgrec          = '/media/macul/black/face_database_raw_data/mscelb_from_insightface/rec/train.rec',
        shuffle              = False,
        rand_mirror          = False,
        mean                 = None,#[127.5,127.5,127.5],#[means["R"],means["G"],means["B"]],
        cutoff               = 0,
        images_filter        = 0,
        aug_list             = [],
    )



In [11]:
train_dataiter

AttributeError: 'MyImageIter_IndividualLabel' object has no attribute 'keys'

In [6]:
batch=train_dataiter.next()

call reset()


In [7]:
batch.label[0]


[[84128.]]
<NDArray 1x1 @cpu(0)>

In [8]:
batch.label[1]


[[-0.01075903  0.01945279 -0.00668122  0.01130166 -0.03332893 -0.01189697
  -0.03152774 -0.04422258  0.0141628  -0.03795503]]
<NDArray 1x10 @cpu(0)>

In [34]:
batch.label[0]


[[84128.]]
<NDArray 1x1 @cpu(0)>

In [10]:
batch.data[0][0,1,:,:]


[[105. 103. 113. ...  82.  78.  79.]
 [107. 111. 124. ...  83.  83.  86.]
 [ 92. 107. 130. ...  83.  86.  92.]
 ...
 [ 83.  80.  71. ... 203. 204. 205.]
 [ 82.  79.  70. ... 208. 207. 205.]
 [ 78.  75.  67. ... 212. 208. 204.]]
<NDArray 112x112 @cpu(0)>

In [10]:
img=(mx.nd.transpose(batch.data[0], axes=(0, 2, 3, 1)).asnumpy()*128+127.5)[0,:,:,:].astype(np.uint8)

In [14]:
img[:,:,1]

array([[105, 103, 113, ...,  82,  78,  79],
       [107, 111, 124, ...,  83,  83,  86],
       [ 92, 107, 130, ...,  83,  86,  92],
       ...,
       [ 83,  80,  71, ..., 203, 204, 205],
       [ 82,  79,  70, ..., 208, 207, 205],
       [ 78,  75,  67, ..., 212, 208, 204]], dtype=uint8)

In [11]:
import cv2
img1 = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
cv2.imshow('test',img1)
cv2.waitKey()

32

In [39]:
np.sum(batch1.data[0][:,:,:]-batch.data[0][0,:,:,:])


[0.]
<NDArray 1 @cpu(0)>

In [37]:
batch.data[0][0,:,:].shape

(3L, 112L, 112L)

In [7]:
batch1.data[0]


[[[-0.44140625 -0.43359375 -0.93359375 ... -0.13671875 -0.08203125
   -0.11328125]
  [-0.41015625 -0.42578125 -0.32421875 ... -0.43359375 -0.33203125
   -0.45703125]
  [-0.30078125 -0.19921875 -0.19140625 ... -0.41796875 -0.98046875
   -0.97265625]
  ...
  [ 0.34765625  0.34765625  0.22265625 ...  0.02734375  0.04296875
    0.07421875]
  [ 0.30859375  0.37109375  0.29296875 ...  0.06640625  0.08984375
    0.09765625]
  [ 0.29296875  0.39453125  0.38671875 ...  0.07421875  0.11328125
    0.12109375]]

 [[-0.48828125 -0.48828125 -0.98828125 ... -0.24609375 -0.19140625
   -0.20703125]
  [-0.46484375 -0.48046875 -0.37890625 ... -0.51953125 -0.39453125
   -0.51171875]
  [-0.33203125 -0.23046875 -0.23046875 ... -0.44140625 -0.98046875
   -0.96484375]
  ...
  [ 0.33203125  0.33203125  0.19140625 ...  0.00390625  0.01953125
    0.05078125]
  [ 0.30859375  0.36328125  0.26171875 ...  0.02734375  0.05078125
    0.05859375]
  [ 0.29296875  0.38671875  0.35546875 ...  0.03515625  0.07421875
    0

In [3]:
sym = fresnet.get_symbol(1000, 50, 
        version_se=0, version_input=1, 
        version_output='E', version_unit=3,
        version_act='prelu')

0 1 E 3 prelu


In [4]:
all_layers = sym.get_internals()

In [5]:
all_layers.list_outputs()

['data',
 'id_output',
 '_minusscalar0_output',
 '_mulscalar0_output',
 'conv0_weight',
 'conv0_output',
 'bn0_gamma',
 'bn0_beta',
 'bn0_moving_mean',
 'bn0_moving_var',
 'bn0_output',
 'relu0_gamma',
 'relu0_output',
 'stage1_unit1_bn1_gamma',
 'stage1_unit1_bn1_beta',
 'stage1_unit1_bn1_moving_mean',
 'stage1_unit1_bn1_moving_var',
 'stage1_unit1_bn1_output',
 'stage1_unit1_conv1_weight',
 'stage1_unit1_conv1_output',
 'stage1_unit1_bn2_gamma',
 'stage1_unit1_bn2_beta',
 'stage1_unit1_bn2_moving_mean',
 'stage1_unit1_bn2_moving_var',
 'stage1_unit1_bn2_output',
 'stage1_unit1_relu1_gamma',
 'stage1_unit1_relu1_output',
 'stage1_unit1_conv2_weight',
 'stage1_unit1_conv2_output',
 'stage1_unit1_bn3_gamma',
 'stage1_unit1_bn3_beta',
 'stage1_unit1_bn3_moving_mean',
 'stage1_unit1_bn3_moving_var',
 'stage1_unit1_bn3_output',
 'stage1_unit1_conv1sc_weight',
 'stage1_unit1_conv1sc_output',
 'stage1_unit1_sc_gamma',
 'stage1_unit1_sc_beta',
 'stage1_unit1_sc_moving_mean',
 'stage1_unit1_sc

In [15]:
sym3 = all_layers['fc1_output']
mod3 = mx.mod.Module(symbol=sym3, label_names=None, context=mx.cpu())
mod3.bind(for_training=False, data_shapes=[('data', (1,3,112,112))])
mod3.init_params()
#mod3.set_params(arg_params, aux_params)

In [16]:

img = get_image('/home/macul/2018-10-15.png')

#output = mod3(Batch([mx.nd.array(img)]))
mod3.forward(Batch([mx.nd.array(img)]))

In [17]:

out = mod3.get_outputs()[0].asnumpy()
print(out.shape)

(1, 1000)


In [17]:
512*7*7

25088

In [ ]:
all_label = mx.symbol.Variable('softmax_label')
gt_label = all_label
extra_loss = None
_weight = mx.symbol.Variable("fc7_weight", shape=(args.num_classes, args.emb_size), lr_mult=args.fc7_lr_mult, wd_mult=args.fc7_wd_mult)

s = args.margin_s #64
m = args.margin_m #0.5
assert s>0.0
assert m>=0.0
assert m<(math.pi/2)
_weight = mx.symbol.L2Normalization(_weight, mode='instance')
nembedding = mx.symbol.L2Normalization(embedding, mode='instance', name='fc1n')*s
fc7 = mx.sym.FullyConnected(data=nembedding, weight = _weight, no_bias = True, num_hidden=args.num_classes, name='fc7')
zy = mx.sym.pick(fc7, gt_label, axis=1)
cos_t = zy/s
cos_m = math.cos(m)
sin_m = math.sin(m)
mm = math.sin(math.pi-m)*m
#threshold = 0.0
threshold = math.cos(math.pi-m)
if args.easy_margin: #0
  cond = mx.symbol.Activation(data=cos_t, act_type='relu')
else:
  cond_v = cos_t - threshold
  cond = mx.symbol.Activation(data=cond_v, act_type='relu')
body = cos_t*cos_t
body = 1.0-body
sin_t = mx.sym.sqrt(body)
new_zy = cos_t*cos_m
b = sin_t*sin_m
new_zy = new_zy - b
new_zy = new_zy*s
if args.easy_margin:
  zy_keep = zy
else:
  zy_keep = zy - s*mm
new_zy = mx.sym.where(cond, new_zy, zy_keep)

diff = new_zy - zy
diff = mx.sym.expand_dims(diff, 1)
gt_one_hot = mx.sym.one_hot(gt_label, depth = args.num_classes, on_value = 1.0, off_value = 0.0)
body = mx.sym.broadcast_mul(gt_one_hot, diff)
fc7 = fc7+body    


out_list = [mx.symbol.BlockGrad(embedding)]
softmax = mx.symbol.SoftmaxOutput(data=fc7, label = gt_label, name='softmax', normalization='valid')
out_list.append(softmax)
if args.loss_type==6:
out_list.append(intra_loss)
if args.loss_type==7:
out_list.append(inter_loss)
#out_list.append(mx.sym.BlockGrad(counter_weight))
#out_list.append(intra_loss)
if args.ce_loss:
#ce_loss = mx.symbol.softmax_cross_entropy(data=fc7, label = gt_label, name='ce_loss')/args.per_batch_size
body = mx.symbol.SoftmaxActivation(data=fc7)
body = mx.symbol.log(body)
_label = mx.sym.one_hot(gt_label, depth = args.num_classes, on_value = -1.0, off_value = 0.0)
body = body*_label
ce_loss = mx.symbol.sum(body)/args.per_batch_size
out_list.append(mx.symbol.BlockGrad(ce_loss))
out = mx.symbol.Group(out_list)

In [12]:
rec_file = '/media/macul/black/face_database_raw_data/ms1m_crop_112x112/rec/train_insight.rec'
idx_file = '/media/macul/black/face_database_raw_data/ms1m_crop_112x112/rec/train_insight.idx'
idx_io = mx.recordio.MXIndexedRecordIO(idx_file, rec_file, 'r')

In [16]:
s = idx_io.read_idx(1)

In [17]:
header, _ = mx.recordio.unpack(s)

In [20]:
header.label

0.0

In [31]:
seq_identity = range(int(header.label[0]), int(header.label[1]))

In [78]:
s = idx_io.read_idx(1)
header, img = mx.recordio.unpack(s)

In [79]:
header

HEADER(flag=11, label=array([ 8.6772000e+04, -1.1738513e-02,  1.1776077e-02,  1.4352672e-02,
        7.2420081e-03, -2.5646670e-03,  5.6658745e-02, -3.0902587e-02,
       -2.5535481e-02,  5.7854794e-02, -2.4363624e-02], dtype=float32), id=1, id2=0)

In [81]:
img = mx.image.imdecode(img)
img.shape

(112L, 112L, 3L)